In [ ]:
Pkg.add("POMDPs")
Pkg.add("POMDPToolbox")

In [1]:
using POMDPs
using POMDPToolbox

In [2]:
importall POMDPs
include("helpers.jl");

# Chain MDP

State space: Integer representing which node we are in.

Action space: Try to move left or right (represented by symbols `:left` and `:right`.

Transitions: Move in action direction with probability `p_success`, otherwise move in the opposite direction.

Reward: +1 for being in rightmost state

In [4]:
plot_chain(4, fill=Dict(4=>"green"))

First, create a new type that represents the MDP

In [6]:
struct ChainMDP <: MDP{Int, Symbol}
    len::Int
    p_success::Float64
    discount::Float64
end

We define the behavior of the MDP by imlementing functions from the POMDPs.jl API (http://juliapomdp.github.io/POMDPs.jl/latest/api/). 

In [7]:
?SparseCat

search: SparseCat sparsevec SparseVector SparseMatrixCSC AbstractSparseVector



```
SparseCat(values, probabilities)
```

Create a sparse categorical distribution.

`values` is an iterable object containing the possible values (can be of any type) in the distribution that have nonzero probability. `probabilities` is an iterable object that contains the associated probabilities.

This is optimized for value iteration with a fast implementation of `weighted_iterator`. Both `pdf` and `rand` are order n.


In [8]:
# the transition function returns the distribution of sp given that
# action a is taken in state s
function transition(mdp::ChainMDP, s::Int, a::Symbol)
    if a == :right
        success = min(s+1, mdp.len)
        failure = max(s-1, 1)
    else # a == :left
        success = max(s-1, 1)
        failure = min(s+1, mdp.len)
    end
    return SparseCat([success, failure], [mdp.p_success, 1.0-mdp.p_success])
end

transition (generic function with 1 method)

Note: this allocates arrays in the return statement - it could be sped up by using [StaticArrays.jl](https://github.com/JuliaArrays/StaticArrays.jl)

In [9]:
function reward(mdp::ChainMDP, s::Int, a::Symbol)
    if s == mdp.len
        return 1.0
    else 
        return 0.0
    end
end

reward (generic function with 2 methods)

In [10]:
discount(mdp::ChainMDP) = mdp.discount

discount (generic function with 4 methods)

And that's all we need to define the MDP (for simulation at least)

In [11]:
mdp = ChainMDP(4, 0.7, 0.9)

ChainMDP(4, 0.7, 0.9)

In [12]:
transition(mdp, 1, :right)

POMDPToolbox.SparseCat{Array{Int64,1},Array{Float64,1}}([2, 1], [0.7, 0.3])

In [13]:
reward(mdp, 1, :right)

0.0

In [14]:
reward(mdp, 4, :left)

1.0

# Simulations

There are three basic ways to simulate a POMDP.



If we want to try out simple policies we can use the `sim` function with a `do` block that defines the policy.

In [17]:
history = sim(mdp, initial_state=1, max_steps=10) do s
    if s == 2
        return :left
    end
    return :right
end;

In [18]:
for (s, a, r) in eachstep(history, "sar")
    @printf("State: %2d    Action: %6s   Reward: %1d\n", s, a, r)
end    

State:  1    Action:  right   Reward: 0
State:  1    Action:  right   Reward: 0
State:  1    Action:  right   Reward: 0
State:  2    Action:   left   Reward: 0
State:  1    Action:  right   Reward: 0
State:  2    Action:   left   Reward: 0
State:  1    Action:  right   Reward: 0
State:  2    Action:   left   Reward: 0
State:  3    Action:  right   Reward: 0
State:  4    Action:  right   Reward: 1


We can also specify a policy object, for example using a vector

In [19]:
@show policy_vector = fill(:right, 4);

policy_vector = fill(:right, 4) = Symbol[:right, :right, :right, :right]


In [20]:
policy = VectorPolicy(mdp, policy_vector);

and step through a simulation as it is being performed

In [21]:
for (s, a, r) in stepthrough(mdp, policy, "sar", initial_state=1, max_steps=10)
    @printf("State: %2d    Action: %6s   Reward: %1d\n", s, a, r)
    sleep(1.0)
end

State:  1    Action:  right   Reward: 0
State:  2    Action:  right   Reward: 0
State:  1    Action:  right   Reward: 0
State:  2    Action:  right   Reward: 0
State:  3    Action:  right   Reward: 0
State:  4    Action:  right   Reward: 1
State:  4    Action:  right   Reward: 1
State:  3    Action:  right   Reward: 0
State:  2    Action:  right   Reward: 0
State:  1    Action:  right   Reward: 0


And also use a simulator object and the `simulate()` function.

In [22]:
simulator = HistoryRecorder(max_steps=10, initial_state=1)
history = simulate(simulator, mdp, policy)
@show state_hist(history);

state_hist(history) = [1, 2, 3, 4, 3, 4, 4, 4, 3, 2, 3]


# Solving with MCTS


The required API functions depend on the solver. Suppose we want to use the MCTS solver from MCTS.jl:

In [23]:
POMDPs.add("MCTS")
using MCTS

Package already installed


INFO: Cloning MCTS from https://github.com/JuliaPOMDP/MCTS.jl


In [24]:
solver = MCTSSolver()
mdp = ChainMDP(4, 0.8, 0.95)

@requirements_info solver mdp


INFO: POMDPs.jl requirements for action(::MCTS.AbstractMCTSPlanner, ::Any) and dependencies. ([✔] = implemented correctly; [X] = missing)

For action(::MCTS.AbstractMCTSPlanner, ::Any):
  [No additional requirements]
For simulate(::MCTS.AbstractMCTSPlanner, ::Any, ::Int64) (in action(::MCTS.AbstractMCTSPlanner, ::Any)):
  [✔] discount(::ChainMDP)
  [✔] isterminal(::ChainMDP, ::Int64)
  [✔] generate_sr(::ChainMDP, ::Int64, ::Symbol, ::MersenneTwister)
  [✔] isequal(::Int64, ::Int64)
  [✔] hash(::Int64)
For insert_node!(::MCTS.AbstractMCTSPlanner, ::Any) (in simulate(::MCTS.AbstractMCTSPlanner, ::Any, ::Int64)):
  [X] actions(::ChainMDP, ::Int64)
For estimate_value(::MCTS.SolvedRolloutEstimator, ::POMDPs.MDP, ::Any, ::Int64) (in simulate(::MCTS.AbstractMCTSPlanner, ::Any, ::Int64)):
  [No additional requirements]
For rollout(::MCTS.SolvedRolloutEstimator, ::POMDPs.MDP, ::Any, ::Int64) (in estimate_value(::MCTS.SolvedRolloutEstimator, ::POMDPs.MDP, ::Any, ::Int64)):
  [No additional requ

LoadError: [91mMethodError: no method matching actions(::ChainMDP)[0m
Closest candidates are:
  actions(::Union{POMDPs.MDP, POMDPs.POMDP}, [91m::Any[39m) at /home/zach/.julia/v0.6/POMDPs/src/space.jl:43
  actions([91m::POMDPs.POMDP{S,Bool,O}[39m) where {S, O} at /home/zach/.julia/v0.6/POMDPToolbox/src/convenience/implementations.jl:6
  actions([91m::POMDPs.POMDP{S,Bool,O}[39m, [91m::S[39m) where {S, O} at /home/zach/.julia/v0.6/POMDPToolbox/src/convenience/implementations.jl:7
  ...[39m

In [25]:
?actions

search: actions n_actions action_hist ordered_actions StateActionStateNode



```
actions(problem::POMDP)
actions(problem::MDP)
```

Returns the entire action space of a POMDP.

```
actions{S,A,O}(problem::POMDP{S,A,O}, state::S)
actions{S,A}(problem::MDP{S,A}, state::S)
```

Return the action space accessible from the given state.

```
actions{S,A,O,B}(problem::POMDP{S,A,O}, belief::B)
```

Return the action space accessible from the states with nonzero belief.


In [26]:
actions(::ChainMDP) = [:left, :right]

actions (generic function with 12 methods)

In [27]:
@requirements_info solver mdp


INFO: POMDPs.jl requirements for action(::MCTS.AbstractMCTSPlanner, ::Any) and dependencies. ([✔] = implemented correctly; [X] = missing)

For action(::MCTS.AbstractMCTSPlanner, ::Any):
  [No additional requirements]
For simulate(::MCTS.AbstractMCTSPlanner, ::Any, ::Int64) (in action(::MCTS.AbstractMCTSPlanner, ::Any)):
  [✔] discount(::ChainMDP)
  [✔] isterminal(::ChainMDP, ::Int64)
  [✔] generate_sr(::ChainMDP, ::Int64, ::Symbol, ::MersenneTwister)
  [✔] isequal(::Int64, ::Int64)
  [✔] hash(::Int64)
For insert_node!(::MCTS.AbstractMCTSPlanner, ::Any) (in simulate(::MCTS.AbstractMCTSPlanner, ::Any, ::Int64)):
  [✔] actions(::ChainMDP, ::Int64)
  [✔] iterator(::Array)
For estimate_value(::MCTS.SolvedRolloutEstimator, ::POMDPs.MDP, ::Any, ::Int64) (in simulate(::MCTS.AbstractMCTSPlanner, ::Any, ::Int64)):
  [No additional requirements]
For rollout(::MCTS.SolvedRolloutEstimator, ::POMDPs.MDP, ::Any, ::Int64) (in estimate_value(::MCTS.SolvedRolloutEstimator, ::POMDPs.MDP, ::Any, ::Int64)

true

In [28]:
solver = MCTSSolver(enable_tree_vis=true)
planner = solve(solver, mdp);

In [29]:
simulate(simulator, mdp, planner, 1)

POMDPToolbox.MDPHistory{Int64,Symbol}([1, 2, 3, 4, 3, 4, 4, 4, 4, 3, 2], Symbol[:right, :right, :right, :right, :right, :right, :right, :right, :right, :right], [0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0], 0.95, Nullable{Exception}(), Nullable{Any}())

In [30]:
using D3Trees
D3Tree(planner, 1)

1
N:    303

├──                     left
│  Q:   0.47
│  N:     12

│  ├──                        2
│  │  N:    697

│  │  ├──                     left
│  │  │  Q:   0.83
│  │  │  N:     30
 (2 children)
│  │  └──                    right
│  │     Q:   2.31
│  │     N:    667
 (2 children)
│  └──                        1
│     N:    303

│     ├──                     left
│     │  Q:   0.47
│     │  N:     12
 (2 children)
│     └──                    right
│        Q:   1.98
│        N:    291
 (2 children)
└──                    right
   Q:   1.98
   N:    291

   ├──                        2
   │  N:    697

   │  ├──                     left
   │  │  Q:   0.83
   │  │  N:     30
 (2 children)
   │  └──                    right
   │     Q:   2.31
   │     N:    667
 (2 children)
   └──                        1
      N:    303

      ├──                     left
      │  Q:   0.47
      │  N:     12
 (2 children)
      └──                    right
         Q:   1.98
         N:    291
 (2 children)

# Solving with value iteration

We can also try value iteration from the DiscreteValueIteration package

In [31]:
POMDPs.add("DiscreteValueIteration")

Package already installed


INFO: Cloning DiscreteValueIteration from https://github.com/JuliaPOMDP/DiscreteValueIteration.jl


In [32]:
using DiscreteValueIteration
solver = ValueIterationSolver()
@requirements_info solver mdp


INFO: POMDPs.jl requirements for solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP}) and dependencies. ([✔] = implemented correctly; [X] = missing)

For solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP}):
  [✔] discount(::ChainMDP)
  [X] n_states(::ChainMDP)
  [X] n_actions(::ChainMDP)
  [✔] transition(::ChainMDP, ::Int64, ::Symbol)
  [✔] reward(::ChainMDP, ::Int64, ::Symbol, ::Int64)
  [✔] state_index(::ChainMDP, ::Int64)
  [X] action_index(::ChainMDP, ::Symbol)
  [✔] actions(::ChainMDP, ::Int64)
For ordered_states(::Union{POMDPs.MDP,POMDPs.POMDP}) (in solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP})):
  [X] states(::ChainMDP)
For ordered_actions(::Union{POMDPs.MDP,POMDPs.POMDP}) (in solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP})):
  [✔] actions(::ChainMDP)
  [✔] iterator(::Array)

Note: Missing methods are often due to incorrect importing. Consider using `importall POMDPs`.

Throwing the first exception (from processing solve(

LoadError: [91mMethodError: no method matching states(::ChainMDP)[0m
Closest candidates are:
  states(::Union{POMDPs.MDP, POMDPs.POMDP}, [91m::Any[39m) at /home/zach/.julia/v0.6/POMDPs/src/space.jl:26
  states([91m::POMDPs.MDP{Bool,A} where A[39m) at /home/zach/.julia/v0.6/POMDPToolbox/src/convenience/implementations.jl:21
  states([91m::POMDPs.POMDP{Bool,A,O} where O where A[39m) at /home/zach/.julia/v0.6/POMDPToolbox/src/convenience/implementations.jl:22[39m

In [33]:
n_states(mdp::ChainMDP) = mdp.len
states(mdp::ChainMDP) = 1:mdp.len
n_actions(::ChainMDP) = 2
action_index(::ChainMDP, a::Symbol) = a == :left ? 1 : 2

action_index (generic function with 3 methods)

In [34]:
@requirements_info solver mdp


INFO: POMDPs.jl requirements for solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP}) and dependencies. ([✔] = implemented correctly; [X] = missing)

For solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP}):
  [✔] discount(::ChainMDP)
  [✔] n_states(::ChainMDP)
  [✔] n_actions(::ChainMDP)
  [✔] transition(::ChainMDP, ::Int64, ::Symbol)
  [✔] reward(::ChainMDP, ::Int64, ::Symbol, ::Int64)
  [✔] state_index(::ChainMDP, ::Int64)
  [✔] action_index(::ChainMDP, ::Symbol)
  [✔] actions(::ChainMDP, ::Int64)
  [✔] iterator(::Array)
  [✔] iterator(::UnitRange)
  [✔] iterator(::SparseCat)
  [✔] pdf(::SparseCat, ::Int64)
For ordered_states(::Union{POMDPs.MDP,POMDPs.POMDP}) (in solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP})):
  [✔] states(::ChainMDP)
For ordered_actions(::Union{POMDPs.MDP,POMDPs.POMDP}) (in solve(::ValueIterationSolver, ::Union{POMDPs.MDP,POMDPs.POMDP})):
  [✔] actions(::ChainMDP)



true

In [35]:
vi_policy = solve(solver, mdp, verbose=true)

[Iteration 1   ] residual:          1 | iteration runtime:      0.050 ms, (  4.99E-05 s total)
[Iteration 2   ] residual:      0.904 | iteration runtime:      0.014 ms, (  6.39E-05 s total)
[Iteration 3   ] residual:      0.839 | iteration runtime:      0.004 ms, (  6.82E-05 s total)
[Iteration 4   ] residual:      0.783 | iteration runtime:      0.004 ms, (  7.21E-05 s total)
[Iteration 5   ] residual:      0.734 | iteration runtime:      0.003 ms, (  7.55E-05 s total)
[Iteration 6   ] residual:      0.688 | iteration runtime:      0.003 ms, (  7.86E-05 s total)
[Iteration 7   ] residual:      0.646 | iteration runtime:      0.003 ms, (  8.19E-05 s total)
[Iteration 8   ] residual:      0.609 | iteration runtime:      0.003 ms, (  8.5E-05 s total)
[Iteration 9   ] residual:      0.576 | iteration runtime:      0.003 ms, (  8.83E-05 s total)
[Iteration 10  ] residual:      0.543 | iteration runtime:      0.003 ms, (  9.14E-05 s total)
[Iteration 11  ] residual:       0.51 | iteration r

DiscreteValueIteration.ValueIterationPolicy([11.8543 12.3175; 12.0587 13.1301; 12.9049 14.1988; 14.7171 15.4018], [12.3175, 13.1301, 14.1988, 15.4018], [2, 2, 2, 2], Symbol[:left, :right], true, ChainMDP(4, 0.8, 0.95))

In [36]:
for s in states(mdp)
    println("State: $s Action: $(action(vi_policy, s))")
end

State: 1 Action: right
State: 2 Action: right
State: 3 Action: right
State: 4 Action: right


# Crying Baby POMDP

In [37]:
Pkg.add("POMDPModels")

INFO: Package POMDPModels is already installed
INFO: METADATA is out-of-date — you may not have the latest version of POMDPModels
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [38]:
using POMDPModels

In [39]:
bp = BabyPOMDP()
up = BabyBeliefUpdater(bp)

POMDPModels.BabyBeliefUpdater(POMDPModels.BabyPOMDP(-5.0, -10.0, 0.1, 0.8, 0.1, 0.9))

In [40]:
POMDPs.add("SARSOP") # might take a while to compile
using SARSOP

INFO: Cloning SARSOP from https://github.com/JuliaPOMDP/SARSOP.jl


Package already installed


In [41]:
solver = SARSOPSolver()

SARSOP.SARSOPSolver(Dict{AbstractString,Any}())

In [42]:
p = solve(solver, bp)

Generating a pomdpx file: model.pomdpx

Loading the model ...
  input file   : model.pomdpx
  loading time : 0.00s 

SARSOP initializing ...
  initialization time : 0.00s

-------------------------------------------------------------------------------
 Time   |#Trial |#Backup |LBound    |UBound    |Precision  |#Alphas |#Beliefs  
-------------------------------------------------------------------------------
 0       0       0        -47.3685   -16.0713   31.2971     2        1        
 0       5       51       -17.9323   -16.0492   1.88318     9        6        
 0       10      103      -16.3978   -16.2159   0.181953    2        8        
 0       14      151      -16.3124   -16.277    0.0353847   2        13       
 0       18      207      -16.3058   -16.2969   0.00887227  2        16       
 0       21      250      -16.3055   -16.3022   0.00329006  7        16       
 0       25      305      -16.3055   -16.3044   0.00111584  2        18       
 0       26      323      -16.3055 

SARSOP.POMDPPolicy("out.policy", POMDPXFiles.POMDPAlphas([-29.6749 -38.2512; -19.6749 -16.3055], [0, 1]), POMDPModels.BabyPOMDP(-5.0, -10.0, 0.1, 0.8, 0.1, 0.9), Any[true, false])

In [43]:
history = simulate(simulator, bp, p, up, BoolDistribution(0.0));

In [44]:
for (b, a) in eachstep(history, "ba")
    @printf("%3.2f hungry belief => %s\n",
            b.p,
            a ? "feed":"don't feed")
end

0.00 hungry belief => don't feed
0.47 hungry belief => feed
0.00 hungry belief => don't feed
0.02 hungry belief => don't feed
0.03 hungry belief => don't feed
0.03 hungry belief => don't feed
0.03 hungry belief => don't feed
0.03 hungry belief => don't feed
0.03 hungry belief => don't feed
0.03 hungry belief => don't feed
